# Interpolation of semi-structured data in five dimensions

According to [this paper], the time complexity of Delauny triangulation is O(...). This is explains why the interpolation cannot work in our case, where we have well over one billion points in five-dimensional space. 

Suppose we have data on an irregular grid in x-y-z space. The grid in x-y is a sheared rectangle; the grid in z is irregularly spaced; we want to linearly interpolate the data onto a regular x-y-z grid. The first option is to run 3D interpolation. The second option is to break the problem into two steps: first interpolate w for each (x, y), then interpolate x-y for each new w. This notebook compares the two approaches.

In [ ]:
import sys
import time
import numpy as np
from scipy import interpolate
from scipy import ndimage
from matplotlib import pyplot as plt
from matplotlib import animation
import proplot as pplt
from tqdm import tqdm, trange

sys.path.append('../')
from tools import utils
from tools import plotting as mplt

pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['savefig.dpi'] = 'figure'
pplt.rc['animation.html'] = 'jshtml'

Define an irregular grid.

In [ ]:
dims = ['x', 'y', 'z']
shape = (32, 32, 32)
xn = np.linspace(-5.0, 5.0, shape[0])
yn = np.linspace(-5.0, 5.0, shape[1])
zn = np.linspace(-5.0, 5.0, shape[2])
Xn, Yn, Zn = np.meshgrid(xn, yn, zn, indexing='ij')

# Mess up grid spacing along z axis.
noise = 0.1
z = np.sort(zn * np.random.uniform(1.0 - noise, 1.0 + noise, shape[2]))
Z = np.zeros(shape)
for i in range(shape[0]):
    for j in range(shape[1]):
        Z[i, j, :] = z

# Apply shearing to x-y grid.
X = Xn + 0.5 * Yn
Y = Yn
coords = [X, Y, Z]

In [ ]:
fig, axes = pplt.subplots(nrows=3, ncols=3, figwidth=5, spanx=False, spany=False)
for i in range(3):
    for j in range(3):
        axes[i, j].scatter(coords[j].ravel(), coords[i].ravel(), s=0.1, color='black')
    axes[i, 0].format(ylabel=dims[i])
    axes[-1, i].format(xlabel=dims[i])
axes.format(suptitle='2D views of the grid', suptitle_kw=dict(fontweight='normal'))
plt.show()

Define a function on the grid.

In [ ]:
def fun(X, Y, Z):
    R = np.sqrt(X**2 + Y**2)
    osc = np.sin(np.pi * 0.1 * (X**2 + Y**2) + Z**2)**2
    return osc * np.exp(-0.5 * (R**2 + X*Z - 0.75*Y*Z))

C = fun(X, Y, Z)

Here are some of the x-y slices as z is varied. Note that the images are 

In [ ]:
fig, ax = pplt.subplots()
ax.format(xlabel='x', ylabel='y', xticks=[], yticks=[])
plt.close()

cbar = None
C_max = np.max(C)
vmax = None

def update(k):
    ax.clear()
    idx = (slice(None), slice(None), k)
    ax.pcolormesh(X[idx].T, Y[idx].T, C[idx].T / C_max, vmax=vmax, ec='None')
    ax.set_title(f'z = {Z[0, 0, k]:.2f}', fontsize='medium')
    
anim = animation.FuncAnimation(fig, update, frames=shape[2])
anim.save('slices.gif', dpi=300)

In [ ]:
fig, axes = pplt.subplots(ncols=5, nrows=2, figwidth=7.0)
for ax, k in zip(axes, range(0, shape[2], 3)):
    idx = (slice(None), slice(None), k)
    ax.pcolormesh(X[idx].T, Y[idx].T, C[idx].T)
    ax.format(xlabel='x', ylabel='y')
    ax.set_title(f'z = {Z[0, 0, k]:.2f}', fontsize='medium')
axes.format(xticks=[], yticks=[])

## 3D interpolation.

Define the interpolation grid.

In [ ]:
new_shape = tuple(np.multiply(shape, [1.5, 1.0, 1.0]).astype(int))
x_new = np.linspace(X.min(), X.max(), new_shape[0])
y_new = np.linspace(Y.min(), Y.max(), new_shape[1])
z_new = np.linspace(Z.min(), Z.max(), new_shape[2])
new_coords = np.meshgrid(x_new, y_new, z_new, indexing='ij')
X_new, Y_new, Z_new = new_coords

Let's first test 2D interpolation on a 2D slice of the 3D function. The new grid is overlayed on the tilted grid on the left. Notice that because the measurement grid is tilted, we need to increase the resolution in interpolation grid along the x axis.

In [ ]:
idx = (slice(None), slice(None), shape[2] // 2)
points = (X[idx].ravel(), Y[idx].ravel())
values = C[idx].ravel()
new_points = (X_new[idx].ravel(), Y_new[idx].ravel())
grid_kws = dict(method='linear', fill_value=0.0)
new_values = interpolate.griddata(points, values, new_points, **grid_kws)

fig, axes = pplt.subplots(ncols=2)
axes[0].pcolormesh(X[idx].T, Y[idx].T, C[idx].T)
axes[1].pcolormesh(X_new[idx].T, Y_new[idx].T, 
                   new_values.reshape(new_shape[0], new_shape[1]).T)
axes[0].scatter(X_new.T, Y_new.T, color='red', ec='None', s=0.1)
axes.format(xlabel='x', ylabel='y', toplabels=['Initial', 'Interpolated'])
plt.show()

Now that the interpolation grid is configured, run the 3D interpolation. 

In [ ]:
grid_kws = dict(method='linear', fill_value=0.0)

start_time = time.time()
points = tuple([U.ravel() for U in coords])
values = C.ravel()
new_points = tuple([U.ravel() for U in new_coords])
new_values = interpolate.griddata(points, values, new_points, **grid_kws)
C_new_3d = new_values.reshape(new_shape)
print(f'{(time.time() - start_time):2f} sec')

## Alternative method 

We don't need to perform the full 3D interpolation; we can take advantage of the original grid structure. First, interpolate over z for each (x, y). Since we will need to resize the array, we will use `_C` as a placeholder.

In [ ]:
_C = np.copy(C)

In [ ]:
C_new = np.zeros((shape[0], shape[1], new_shape[2]))

In [ ]:
start_time = time.time()
new_points = z_new
for i in range(shape[0]):
    for j in range(shape[1]):
        points = Z[i, j, :]
        values = _C[i, j, :]
        C_new[i, j, :] = interpolate.griddata(points, values, new_points, **grid_kws)
_C = np.copy(C_new)
print(f'{(time.time() - start_time):2f} sec')

We can view the results for different x and y pixels.

In [ ]:
i = shape[0] // 2
fig, axes = pplt.subplots(nrows=2, ncols=4, figwidth=6)
for ax, j in zip(axes, range(shape[1] // 4, 3 * shape[1] // 4, 1)):
    idx = (i, j, slice(None))
    ax.plot(Z[i, j, :], C[i, j, :], marker='.', ms=3, lw=0, color='black', label='data')
    ax.plot(z_new, C_new[i, j, :], color='blue7', alpha=0.3, label='int')
    ax.plot(z_new, C_new_3d[i, j, :], color='pink7', alpha=0.3, label='int3d')
    ax.annotate(f'i,j = ({i},{j})', xy=(0.02, 0.98), xycoords='axes fraction',
                fontsize='small', verticalalignment='top')
axes[0, -1].legend(loc='t', ncols=1, framealpha=0)
axes.format(xlabel='z')
plt.show()

I don't know why the 3D method is so bad?

We now need to make new coordinate arrays.

In [ ]:
X = utils.copy_into_new_dim(X[:, :, 0], new_shape[2], axis=-1)
Y = utils.copy_into_new_dim(Y[:, :, 0], new_shape[2], axis=-1)
Z = utils.copy_into_new_dim(z_new, (shape[0], shape[1]), axis=0)

In [ ]:
print(X.shape, Y.shape, Z.shape)

Now interpolate x-y for each z.

In [ ]:
start_time = time.time()
C_new = np.zeros(new_shape)
new_points = tuple([U.ravel() for U in np.meshgrid(x_new, y_new, indexing='ij')])
for k in range(new_shape[2]):
    points = (X[:, :, k].ravel(), Y[:, :, k].ravel())
    values = _C[:, :, k].ravel()
    new_values = interpolate.griddata(points, values, new_points, **grid_kws)
    C_new[:, :, k] = new_values.reshape(new_shape[0], new_shape[1])
print(f'{(time.time() - start_time):2f} sec')

Below is a plot of some x-y slices.

In [ ]:
kk = np.arange(0, new_shape[2], 4)

fig, axes = pplt.subplots(ncols=6, nrows=3, figwidth=7.0, space=0.5)
for row, array in enumerate([C, C_new, C_new_3d]):
    for ax, k in zip(axes[row, :], kk):
        idx = (slice(None), slice(None), k)
        if row == 0:
            x, y = X[idx].T, Y[idx].T
        else:
            x, y = x_new, y_new
        ax.pcolormesh(x, y, array[idx].T)
        if row == 0:
            ax.set_title(f'z = {z_new[k]:.2f}', fontsize='medium')
axes.format(xticks=[], yticks=[], xlabel='x', ylabel='y',
            rowlabels=['C_meas', 'C_new', 'C_new_3d'])